# CNN


## import libraries

In [1]:
import os
import json
from PIL import Image

import torch
import torch.utils.data as data
import torchvision.transforms.v2 as v2
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

## dataset

### unzip

In [2]:
!unzip /content/dataset_gen_reg.zip

Archive:  /content/dataset_gen_reg.zip
   creating: dataset_gen_reg/
   creating: dataset_gen_reg/images/
  inflating: dataset_gen_reg/images/back_1.png  
  inflating: dataset_gen_reg/images/back_10.png  
  inflating: dataset_gen_reg/images/back_2.png  
  inflating: dataset_gen_reg/images/back_3.png  
  inflating: dataset_gen_reg/images/back_4.png  
  inflating: dataset_gen_reg/images/back_5.png  
  inflating: dataset_gen_reg/images/back_6.png  
  inflating: dataset_gen_reg/images/back_7.png  
  inflating: dataset_gen_reg/images/back_8.png  
  inflating: dataset_gen_reg/images/back_9.png  
 extracting: dataset_gen_reg/images/sun64.png  


In [3]:
import shutil

In [4]:
shutil.move('/content/dataset_gen_reg/images', '/content')

'/content/images'

In [5]:
os.remove("/content/dataset_gen_reg.zip")

In [6]:
os.rmdir('/content/dataset_gen_reg')

### data

In [7]:
train_data = {'total': 10000, 'dir': "train"}
test_data = {'total': 1000, 'dir': "test"}
total_bk = 10
total_cls = 4
dir_out = 'dataset_reg'
file_format = 'format.json'
cls = [(255, 255, 255), (0, 0, 255), (0, 255, 0), (255, 0, 0)]

libraries

In [8]:
from random import randint
import json
import pygame

pygame 2.6.1 (SDL 2.28.4, Python 3.12.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


folders

In [9]:
if not os.path.exists(dir_out):
    os.mkdir(dir_out)
    if not os.path.exists(os.path.join(dir_out, "train")):
        os.mkdir(os.path.join(dir_out, "train"))
    if not os.path.exists(os.path.join(dir_out, "test")):
        os.mkdir(os.path.join(dir_out, "test"))

In [10]:
sun = pygame.image.load("images/sun64.png")
backs = [pygame.image.load(f"images/back_{n}.png") for n in range(1, total_bk+1)]

image

In [11]:
for info in (train_data, test_data):
    sun_coords = dict()

    for i in range(1, info['total']+1):
        file_out = f"sun_reg_{i}.png"
        im = backs[randint(0, total_bk-1)].copy()

        for _ in range(randint(20, 100)):
            x0 = randint(0, 256)
            y0 = randint(0, 256)
            pygame.draw.circle(im, cls[randint(0, total_cls-1)], (x0, y0), 1)

        x = randint(32, 256 - 32)
        y = randint(32, 256 - 32)
        sun_coords[file_out] = (x, y)
        im.blit(sun, (x-32, y-32))

        pygame.image.save(im, os.path.join(dir_out, info['dir'], file_out))

    fp = open(os.path.join(dir_out, info['dir'], file_format), "w")
    json.dump(sun_coords, fp)
    fp.close()

## class Dataset

In [12]:
class SunData(data.Dataset):
  def __init__(self, path, train= True, transform=None) -> None:
      super().__init__()
      self.path = os.path.join('/content/dataset_reg', 'train' if train else 'test')
      self.transform = transform

      with open(os.path.join(self.path, 'format.json'), 'r') as f:
        self.format = json.load(f)

      self.length = len(self.format)
      self.files = tuple(self.format.keys())
      self.targets = tuple(self.format.values())

  def __getitem__(self, item):
      path_file = os.path.join(self.path, self.files[item])
      img = Image.open(path_file).convert('RGB')

      if self.transform:
        img = self.transform(img)

      return img, torch.tensor(self.targets[item], dtype=torch.float32)

  def __len__(self):
    return self.length

In [13]:
transorm = v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])
batch_size = 150


d_train = SunData('/content/dataset_reg', transform=transorm)
train_loader = data.DataLoader(d_train, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=8)
d_test = SunData('/content/dataset_reg', train=False,transform=transorm)
test_loader = data.DataLoader(d_test, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=8)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


##sun CNN

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [15]:
model = nn.Sequential(
    nn.Conv2d(3, 32, 3, padding='same'),
    nn.ReLU(),
    nn.MaxPool2d(2), #batch 32 128 128

    nn.Conv2d(32, 16, 3, padding='same'),
    nn.ReLU(),
    nn.MaxPool2d(2), #batch 8 64 64

    nn.Conv2d(16, 8, 3, padding='same'),
    nn.ReLU(),
    nn.MaxPool2d(2), #batch 4 32 32

    nn.Flatten(),
    nn.Linear(32*32*8, 128),
    nn.ReLU(),
    nn.Linear(128, 2),
)

## train

In [16]:
optimizer = optim.Adam(params=model.parameters(), lr=0.001, weight_decay=0.001) # L2
loss_f = nn.MSELoss()

In [17]:
epoch = 10
model.to(device)
model.train()


for _e in range(epoch):
  loss_mean = 0
  lm_count = 0

  training_tqdm = tqdm(train_loader, leave=True)
  for x_train, y_train in training_tqdm:
    x_train = x_train.to(device, non_blocking=True)
    y_train = y_train.to(device, non_blocking=True)
    pred = model(x_train)
    loss = loss_f(pred, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    lm_count += 1
    loss_mean = 1/lm_count*loss.item() + (1-1/lm_count)*loss_mean


100%|██████████| 67/67 [00:31<00:00,  2.12it/s]


## test

In [19]:
model.eval()
Q=0
count=1

testing_tqdm = tqdm(test_loader, leave=True)
for x_test, y_test in testing_tqdm:
  with torch.no_grad():
    x_test = x_test.to(device)
    y_test = y_test.to(device)
    pred = model(x_test)
    Q += loss_f(pred, y_test).item()
    count += 1

Q/count

100%|██████████| 7/7 [00:05<00:00,  1.31it/s]


7.0555614829063416